In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/events.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [3]:
users = df[['person']].drop_duplicates().copy()

In [4]:
df.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version'],
      dtype='object')

In [5]:
#Storage más buscado


In [6]:
df['event'].head(5)

0    viewed product
1    viewed product
2    viewed product
3    viewed product
4    viewed product
Name: event, dtype: object

In [ ]:
df = df

In [7]:
df_maxs = df.groupby('person')['storage'].value_counts()
df_most_events = df_maxs.rename(columns={'storage':'max'}).reset_index().groupby('person').first()[['storage']].reset_index()
df_most_events.rename(columns={'storage':'storage_mas_eventos'}, inplace=True)

In [8]:
df_views = df.loc[df['event'] == 'viewed product', :]
df_maxs = df_views.groupby('person')['storage'].value_counts()
df_mostView = df_maxs.rename(columns={'storage':'max'}).reset_index().groupby('person').first()[['storage']].reset_index()

In [9]:
df_mostView.head(10)

,person,storage
0,00091926,32GB
1,00091a7a,128GB
2,000ba417,16GB
3,000c79fe,128GB
4,000e4d9e,32GB
5,000e619d,32GB
6,001001be,16GB
7,0010e89a,8GB
8,0016c4b5,32GB
9,001802e4,256GB


In [10]:
df_checkout = df.loc[df['event'] == 'checkout', :]
df_maxs = df_checkout.groupby('person')['storage'].value_counts()
df_mostCheckout = df_maxs.rename(columns={'storage':'max'}).reset_index().groupby('person').first()[['storage']].reset_index()

In [11]:
df_mostCheckout.rename(columns={'storage':'storage_mas_checkout'}, inplace=True)

In [12]:
# res_final = res_final[['storage']]
# res_final.fillna('Unknown', inplace=True)
storages_info_df = pd.merge(users,df_mostView, left_on = 'person', right_on= 'person', how = 'left').set_index('person')
storages_info_df.rename(columns={'storage':'storage_mas_visto'}, inplace=True)

In [13]:
storages_info_df = pd.merge(storages_info_df,df_mostCheckout, left_index=True, right_on= 'person', how = 'left').set_index('person')

In [14]:
storages_info_df = pd.merge(storages_info_df,df_most_events, left_index=True, right_on= 'person', how = 'left').set_index('person')

In [15]:

mymap = {'Nan':0, '256GB':1, '128GB':2, '64GB':3, '32GB':4 , '16GB':5, '8GB':6, '4GB':7, '512MB':8}
storages_info_df = storages_info_df.applymap(lambda s: mymap.get(s) if s in mymap else s)


In [16]:
storages_info_df.fillna(0, inplace=True)
storages_info_df.to_csv('../data/feature_dispositivos_storage.csv', sep=',')